In [ ]:
# Link to the Github repository
# https://github.com/zju3dv/LoFTR

In [1]:
# Configure environment and grab LoFTR code.
!rm -rf sample_data
!pip install torch einops yacs kornia
!git clone https://github.com/zju3dv/LoFTR --depth 1
!mv LoFTR/* . && rm -rf LoFTR

# Download pretrained weights
!mkdir weights 
%cd weights/
!gdown --id 1M-VD35-qdB5Iw-AtbDBCKC7hPolFW9UY  # outdoor-ds
%cd ..

     |████████████████████████████████| 512 kB 8.2 MB/s 
Cloning into 'LoFTR'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 173 (delta 26), reused 142 (delta 21), pack-reused 0
Receiving objects: 100% (173/173), 26.29 MiB | 26.55 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/weights
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1M-VD35-qdB5Iw-AtbDBCKC7hPolFW9UY
To: /content/weights/outdoor_ds.ckpt
100% 46.3M/46.3M [00:01<00:00, 38.5MB/s]
/content


In [2]:
import torch
import cv2
import numpy as np
import matplotlib.cm as cm
from src.utils.plotting import make_matching_figure
from src.loftr import LoFTR, default_cfg

In [3]:
# The default config uses dual-softmax.
# The outdoor and indoor models share the same config.
# You can change the default values like thr and coarse_match_type.
matcher = LoFTR(config=default_cfg)
matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.eval().cuda()

In [4]:
# Upload Images.zip folder and then run this cell
!unzip /content/Images.zip -d /content/new_images

Archive:  /content/Images.zip
  inflating: /content/new_images/Test/test_cattle_1.jpg  
  inflating: /content/new_images/Test/test_cattle_5.jpg  
  inflating: /content/new_images/Database/database_cattle_18.jpg  
  inflating: /content/new_images/Database/database_cattle_9.jpg  
  inflating: /content/new_images/Database/database_cattle_5.jpg  
  inflating: /content/new_images/Database/database_cattle_15.jpg  
  inflating: /content/new_images/Database/database_cattle_19.jpg  
  inflating: /content/new_images/Test/test_cattle_3.jpg  
  inflating: /content/new_images/Test/test_cattle_4.jpg  
  inflating: /content/new_images/Database/database_cattle_6.jpg  
  inflating: /content/new_images/Database/database_cattle_4.jpg  
  inflating: /content/new_images/Test/test_cattle_2.jpg  
  inflating: /content/new_images/Database/database_cattle_11.jpg  
  inflating: /content/new_images/Database/database_cattle_10.jpg  
  inflating: /content/new_images/Database/database_cattle_1.jpg  
  inflating: /c

In [5]:
# Rerun this cell (and below) if a new image pair is uploaded.
import glob
search_dict = {}
for i in glob.glob('/content/new_images/Test/*.jpg'):
  scores = []
  data_names = []
  for z in glob.glob('/content/new_images/Database/*.jpg'):
    img0_raw = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    img1_raw = cv2.imread(z, cv2.IMREAD_GRAYSCALE)
    img0_raw = cv2.resize(img0_raw, (640, 480))
    img1_raw = cv2.resize(img1_raw, (640, 480))

    img0 = torch.from_numpy(img0_raw)[None][None].cuda() / 255.
    img1 = torch.from_numpy(img1_raw)[None][None].cuda() / 255.
    batch = {'image0': img0, 'image1': img1}

    # Inference with LoFTR and get prediction
    with torch.no_grad():
        matcher(batch)
        mkpts0 = batch['mkpts0_f'].cpu().numpy()
        mkpts1 = batch['mkpts1_f'].cpu().numpy()
        mconf = batch['mconf'].cpu().numpy()
    scores.append(len(mkpts0))
    data_names.append(z)
  search_dict[i] = scores

/content/src/loftr/utils/coarse_matching.py:246: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  [i_ids % data['hw0_c'][1], i_ids // data['hw0_c'][1]],
/content/src/loftr/utils/coarse_matching.py:249: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  [j_ids % data['hw1_c'][1], j_ids // data['hw1_c'][1]],
/usr/local/lib/python3.7/dist-packag

In [6]:
import pandas as pd
test = []
Database = []
for key, value in search_dict.items():
  new_key = key.replace('/content/new_images/Test/','').split('.jpg')[0]
  max_score_name = data_names[value.index(max(value))].replace('/content/new_images/Database/','').split('.jpg')[0]
  print(new_key,max_score_name,max(value))
  test.append(new_key)
  if max(value) > 300:
    Database.append(max_score_name)
  else:
    Database.append('Not Found')
df = pd.DataFrame(list(zip(test, Database)),
               columns =['Test Image', 'Database Image'])

test_cattle_1 database_cattle_3 515
test_cattle_3 database_cattle_13 618
test_cattle_5 database_cattle_20 895
test_cattle_4 database_cattle_5 284
test_cattle_2 database_cattle_8 384


In [7]:
df.to_csv('out_loftr.csv',index=False)

In [8]:
df

,Test Image,Database Image
0,test_cattle_1,database_cattle_3
1,test_cattle_3,database_cattle_13
2,test_cattle_5,database_cattle_20
3,test_cattle_4,Not Found
4,test_cattle_2,database_cattle_8


In [10]:
from google.colab import files
files.download("out_loftr.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>